In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub


D:\chientuhocai\Traffic_Sign\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!pip list

In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

tcc3281_vietnametrafficsignclassification_path = kagglehub.dataset_download('tcc3281/vietnametrafficsignclassification')

print('Data source import complete.')
tcc3281_vietnametrafficsignclassification_path


In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import numpy as np
import seaborn as sns
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [5]:
EPOCHS = 30
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
NUM_CLASSES = 51

# 1. Loading and Processing Data

In [ ]:
tcc3281_vietnametrafficsignclassification_path=f'D:\\chientuhocai\\Revise_data\\classes'

In [6]:
# TRAIN_PATH='/kaggle/input/vietnametrafficsignclassification/dataset/train'
# VAL_PATH='/kaggle/input/vietnametrafficsignclassification/dataset/valid'
# TEST_PATH='/kaggle/input/vietnametrafficsignclassification/dataset/test'
# CLASS_MAPPING_PATH='/kaggle/input/vietnametrafficsignclassification/dataset/class_mapping.txt'
TRAIN_PATH=f'{tcc3281_vietnametrafficsignclassification_path}\\dataset\\train'
VAL_PATH=f'{tcc3281_vietnametrafficsignclassification_path}\\dataset\\valid'
TEST_PATH=f'{tcc3281_vietnametrafficsignclassification_path}\\dataset\\test'
CLASS_MAPPING_PATH=f'{tcc3281_vietnametrafficsignclassification_path}\\class_mapping.txt'


NameError: name 'tcc3281_vietnametrafficsignclassification_path' is not defined

In [7]:
# Hàm đọc class mapping từ file txt
def load_class_mapping(txt_path):
    idx_to_class = {}
    with open(txt_path, 'r') as f:
        for line in f:
            idx, class_name = line.strip().split(':')
            idx_to_class[int(idx)] = class_name.strip()
    return idx_to_class

# Load mapping
idx_to_class = load_class_mapping(CLASS_MAPPING_PATH)
class_to_idx = {v: k for k, v in idx_to_class.items()}  # Đảo ngược để có class_to_idx

NameError: name 'CLASS_MAPPING_PATH' is not defined

In [8]:
class_to_idx

NameError: name 'class_to_idx' is not defined

In [8]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
# Load dataset
train_dataset = datasets.ImageFolder(TRAIN_PATH, transform=train_transform)
print('Load train dataset done!')
val_dataset = datasets.ImageFolder(VAL_PATH, transform=val_transform)
print('Load valid dataset done')
test_dataset = datasets.ImageFolder(TEST_PATH, transform=test_transform)
print('Load test dataset done')

In [10]:
# Ghi đè class names bằng mapping từ file
train_dataset.classes = [idx_to_class[i] for i in sorted(idx_to_class.keys())]
train_dataset.class_to_idx = {cls: i for i, cls in enumerate(train_dataset.classes)}

# Ghi đè class names bằng mapping từ file
val_dataset.classes = [idx_to_class[i] for i in sorted(idx_to_class.keys())]
val_dataset.class_to_idx = {cls: i for i, cls in enumerate(val_dataset.classes)}

# Ghi đè class names bằng mapping từ file
test_dataset.classes = [idx_to_class[i] for i in sorted(idx_to_class.keys())]
test_dataset.class_to_idx = {cls: i for i, cls in enumerate(test_dataset.classes)}

# Tạo DataLoader (giữ nguyên)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Load train loader done!')
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
print('Load valid loader done!')
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
print('Load test loader done!')

# 2. Model

In [11]:
# Khởi tạo model
model = models.resnet50(pretrained=True)

# Chiến lược đóng băng layer
# ----------------------------
# Phương án 1: Đóng băng nhiều hơn (phù hợp nếu dữ liệu ít)
for name, param in model.named_parameters():
    if "layer3" not in name and "layer4" not in name and "fc" not in name:
        param.requires_grad = False

# Phương án 2: Mở thêm một số layer (nếu muốn model linh hoạt hơn)
# for name, param in model.named_parameters():
#     if "layer2" not in name and "layer3" not in name and "layer4" not in name and "fc" not in name:
#         param.requires_grad = False

# Sửa đổi lớp fully connected
# ----------------------------
model.fc = nn.Sequential(
    nn.Dropout(0.6),  # Tăng dropout từ 0.5 -> 0.6
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.4),  # Thêm dropout layer thứ 2
    nn.Linear(512, NUM_CLASSES)
)

# In số lượng parameter sẽ được training
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {total_params:,}")

In [12]:
# Chuyển model sang GPU (nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [13]:
criterion = torch.nn.CrossEntropyLoss()
# Thêm weight decay và điều chỉnh learning rate
optimizer = torch.optim.AdamW([
    {'params': model.layer3.parameters(), 'lr': 5e-5, 'weight_decay': 1e-5},
    {'params': model.layer4.parameters(), 'lr': 5e-5, 'weight_decay': 1e-5},
    {'params': model.fc.parameters(), 'lr': 1e-4, 'weight_decay': 1e-5}  # Giảm lr fc từ 1e-3 -> 1e-4
])

# Nên dùng learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [14]:
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score
import math

# Lists để lưu metrics
epochs_list = []
train_loss_list = []
val_loss_list = []
accuracy_list = []
precision_list = []
recall_list = []
early_stop_patience = 5  # Tăng patience từ 3 -> 5
best_val_loss = math.inf  # Theo dõi val loss tốt nhất
patience = early_stop_patience  # Khởi tạo patience

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0

    # === Training Phase ===
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]", leave=True)
    for inputs, labels in train_loop:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        # Trong training loop, sau loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_loop.set_postfix(loss=loss.item())

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []

    val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]", leave=True)
    with torch.no_grad():
        for inputs, labels in val_loop:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            val_loop.set_postfix(val_loss=loss.item())

    # === Tính toán metrics ===
    train_loss /= len(train_dataset)
    val_loss /= len(val_dataset)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')

    # Lưu metrics
    epochs_list.append(epoch + 1)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)

    # In thông tin epoch
    print(f"\nEpoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Accuracy: {accuracy:.4f} | "
          f"Precision: {precision:.4f} | "
          f"Recall: {recall:.4f}")

    # === Early Stopping Logic ===
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "resnet50_best_model_v04.pth")  # Lưu model tốt nhất
        patience = early_stop_patience  # Reset patience khi có cải thiện
        print(f"🌟 Model improved (val_loss: {val_loss:.4f}), saving model...")
    else:
        patience -= 1
        print(f"⏳ Patience decreased to {patience} (val_loss: {val_loss:.4f})")

        if patience <= 0:
            print("🛑 Early stopping triggered!")
            break  # Dừng training nếu hết patience

    scheduler.step()

In [15]:
torch.save(model.state_dict(), "resnet50_model_v04.pth")

# 3. Evaluation

In [4]:
# Lấy danh sách tên lớp từ idx_to_class (đã load từ file mapping)
class_names = [idx_to_class[i] for i in sorted(idx_to_class.keys())]

# Ví dụ output:
# ['speed_limit_20', 'speed_limit_30', ..., 'stop']

NameError: name 'idx_to_class' is not defined

In [17]:
plt.figure(figsize=(12, 4))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(epochs_list, train_loss_list, label='Train Loss')
plt.plot(epochs_list, val_loss_list, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()

# Plot Metrics
plt.subplot(1, 2, 2)
plt.plot(epochs_list, accuracy_list, label='Accuracy')
plt.plot(epochs_list, precision_list, label='Precision (Macro)')
plt.plot(epochs_list, recall_list, label='Recall (Macro)')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Model Performance Metrics')
plt.legend()

plt.tight_layout()
plt.show()

In [18]:
metrics_df = pd.DataFrame({
    'Epoch': epochs_list,
    'Train Loss': train_loss_list,
    'Val Loss': val_loss_list,
    'Accuracy': accuracy_list,
    'Precision': precision_list,
    'Recall': recall_list
})

metrics_df.to_csv('training_metrics_v04.csv', index=False)

In [19]:
metrics_df

In [20]:
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Evaluating Test Set',leave=True):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


In [21]:
def plot_confusion_matrix(y_true, y_pred, classes, normalize=False, title=None, cmap='Blues', figsize=(12, 10), fontsize=12, output_file=None):
    """
    Vẽ confusion matrix được định dạng đẹp mắt và lưu ra file.

    Parameters:
    y_true (array): Nhãn thực tế
    y_pred (array): Nhãn dự đoán
    classes (list): Danh sách tên các lớp
    normalize (bool): Có chuẩn hóa theo hàng hay không
    title (str): Tiêu đề đồ thị
    cmap (str): Bảng màu
    figsize (tuple): Kích thước figure
    fontsize (int): Cỡ chữ
    output_file (str): Đường dẫn file để lưu hình ảnh
    """
    # Tính confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Chuẩn hóa nếu được yêu cầu
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = '.2f'
    else:
        fmt = 'd'

    # Tạo figure
    plt.figure(figsize=figsize, dpi=100)

    # Tạo heatmap
    ax = sns.heatmap(
        cm,
        annot=True,
        fmt=fmt,
        cmap=cmap,
        cbar=False,
        square=True,
        annot_kws={'size': fontsize-2},
        linewidths=0.5,
        linecolor='lightgray'
    )

    # Thiết lập tiêu đề
    if title is None:
        title = 'Normalized Confusion Matrix' if normalize else 'Confusion Matrix'
    ax.set_title(title, fontsize=fontsize+2, pad=20)

    # Thiết lập nhãn trục
    ax.set_xlabel('Predicted Label', fontsize=fontsize)
    ax.set_ylabel('True Label', fontsize=fontsize)

    # Thiết lập tick labels
    tick_marks = np.arange(len(classes)) + 0.5
    ax.set_xticks(tick_marks)
    ax.set_yticks(tick_marks)
    ax.set_xticklabels(classes, rotation=45, ha='right', fontsize=fontsize-1)
    ax.set_yticklabels(classes, rotation=0, fontsize=fontsize-1)

    # Điều chỉnh layout
    plt.tight_layout()

    # Thêm colorbar riêng để kiểm soát tốt hơn
    cax = plt.gcf().add_axes([ax.get_position().x1+0.01, ax.get_position().y0, 0.02, ax.get_position().height])
    plt.colorbar(ax.collections[0], cax=cax)

    # Hiển thị grid nhẹ
    ax.grid(False)
    ax.hlines(y=np.arange(len(classes)+1), xmin=0, xmax=len(classes), colors='white', linewidths=0.5)
    ax.vlines(x=np.arange(len(classes)+1), ymin=0, ymax=len(classes), colors='white', linewidths=0.5)

    # Lưu hình ảnh ra file nếu được yêu cầu
    if output_file:
        plt.savefig(output_file)



In [22]:
# Gọi hàm đánh giá
test_labels, test_preds = evaluate_model(model, test_loader, device)

# In báo cáo classification
print("\nTest Set Evaluation:")
print(classification_report(test_labels, test_preds, target_names=class_names))

In [23]:
plot_confusion_matrix(
    test_labels,
    test_preds,
    classes=class_names,
    normalize=True,
    title='Biểu đồ Test Confusion Matrix (Chuẩn hóa theo hàng)',
    cmap='YlOrBr',
    figsize=(20, 18),  # Tăng kích thước figure
    fontsize=8,        # Giảm kích thước font chữ
    output_file='test_confusion_matrix_large.png'
)
plt.show()

In [24]:
# Gọi hàm đánh giá
valid_labels, valid_preds = evaluate_model(model, val_loader, device)

# In báo cáo classification
print("\Valid Set Evaluation:")
print(classification_report(valid_labels, valid_preds, target_names=class_names))

In [25]:
# Sử dụng
plot_confusion_matrix(
    valid_labels,
    valid_preds,
    classes=class_names,
    normalize=True,
    title='Biểu đồ Valid Confusion Matrix (Chuẩn hóa theo hàng)',
    cmap='YlOrBr',
    figsize=(20, 18),  # Tăng kích thước figure
    fontsize=8,        # Giảm kích thước font chữ
    output_file='valid_confusion_matrix_large.png'
)
plt.show()

In [26]:
# Gọi hàm đánh giá
train_labels, train_preds = evaluate_model(model, train_loader, device)

# In báo cáo classification
print("\Train Set Evaluation:")
print(classification_report(train_labels, train_preds, target_names=class_names))

In [27]:
# Sử dụng
plot_confusion_matrix(
    train_labels,
    train_preds,
    classes=class_names,
    normalize=True,
    title='Biểu đồ Train Confusion Matrix (Chuẩn hóa theo hàng)',
    cmap='YlOrBr',
    figsize=(20, 18),  # Tăng kích thước figure
    fontsize=8,        # Giảm kích thước font chữ
    output_file='train_confusion_matrix_large.png'
)
plt.show()

In [ ]:
# tạo confidence score cho từng lớp
